### capstone project week 3 1&2/3

#### importing libraries

In [9]:
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#### web scraping

In [10]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
src = result.content
soup = BeautifulSoup(src, 'html.parser')

In [11]:
#print(soup.prettify())
postalcode_html = soup.find_all('table')[0]

In [12]:
table_rows = postalcode_html.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
postalcode_table = pd.DataFrame(l, columns=column_names)

In [13]:
# drop rows with no values in Borough
postalcode_table.dropna(subset=['Borough'], axis=0, inplace=True)    

# get indexes for which Borough column has 'Not assigned', then delete rows from dataframe
indexNoBorough = postalcode_table[ postalcode_table['Borough'] == 'Not assigned' ].index
postalcode_table.drop(indexNoBorough , inplace=True)

# reset index after dropped rows
postalcode_table.reset_index(drop=True, inplace=True)
postalcode_table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [14]:
# combine Neighborhood based on same Postalcode and Borough
postalcode_table = postalcode_table.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ','.join(x))
postalcode_table.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


#### Shape of postal code

In [15]:
postalcode_table.shape

(103, 3)

In [19]:
path = "http://cocl.us/Geospatial_data"
df = pd.read_csv(path)
df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
print(df.shape)

(103, 3)


In [26]:
postalcd_Lat_Lng_table = postalcode_table.join(df, how='outer')
postalcd_Lat_Lng_table.head()

del postalcd_Lat_Lng_table['Postal Code']
postalcd_Lat_Lng_table.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


#### Shape of joined table

In [27]:
postalcd_Lat_Lng_table.shape

(103, 5)